# Neo4j 그래프 데이터베이스 구축

뉴스 기사 데이터를 Neo4j 그래프 데이터베이스에 적재

## 그래프 구조
- **Article** 노드: 기사 정보 (article_id, title, url, published_date)
- **Content** 노드: 기사 내용 청크 (chunk, article_id, title, url, published_date, chunk_index)
- **Media** 노드: 언론사 정보 (name)
- **Category** 노드: 카테고리 정보 (name)

## 관계
- Article -[HAS_CHUNK]-> Content
- Media -[PUBLISHED]-> Article  
- Article -[BELONGS_TO]-> Category

In [1]:
import pandas as pd
df = pd.read_excel('Articles_20251121_133658.xlsx')
df.head()

,article_id,title,content,url,published_date,source,author,category
0,ART_018_0006168585,"北, 사업 성공한 부부 ‘공개 처형’…“태도 오만해”","북한 개인 사업 운영 50대 부부,\n“오만해졌다” 야외 공간서 처형\n당국 “대중...",https://n.news.naver.com/mnews/article/018/000...,2025-11-20 14:42:17,이데일리,권혜미 기자,정치
1,ART_421_0008616962,"李대통령, 남아공서 프랑스·독일과 정상회담…""국제정세 변화 대응""","G20 계기 유럽 핵심 강국 佛·獨 양자회담 성사\n""MIKTA 회동서 다자주의·국...",https://n.news.naver.com/mnews/article/421/000...,2025-11-21 03:37:20,뉴스1,심언기 기자,정치
2,ART_366_0001124550,‘마이웨이’ 법사위에 뿔난 與 지도부… “대통령 외교 성과 또 빛바랠라’”,김병기 “협의했어야… 뒷감당은 법사위가”\n\n더불어민주당 지도부와 국회 법제사법위...,https://n.news.naver.com/mnews/article/366/000...,2025-11-20 11:55:09,조선비즈,박숙현 기자,정치
3,ART_052_0002276132,"이 대통령 ""남북 대결 끝내고 평화 공존의 시대로...단계적 비핵화 추진""","이집트를 공식 방문 중인 이재명 대통령은 남북 적대와 대결의 시대를 끝내고, 평화 ...",https://n.news.naver.com/mnews/article/052/000...,2025-11-21 03:06:56,YTN,홍민기 기자,정치
4,ART_449_0000327216,이 대통령 지지율 60%…민주 43%·국힘 24%,이집트를 공식 방문 중인 이재명 대통령과 김혜경 여사가 20일(현지 시간) 카이로 ...,https://n.news.naver.com/mnews/article/449/000...,2025-11-21 10:52:16,채널A,윤승옥 기자 touch@ichannela.com,정치


In [2]:
import neo4j
import os

import dotenv
dotenv.load_dotenv()

URI = os.getenv("NEO4J_URI", "neo4j://localhost:7687")
AUTH = ("neo4j", os.getenv("NEO4J_PASSWORD", "password"))

driver = neo4j.GraphDatabase.driver(URI, auth=AUTH)

In [3]:
def chunk_text(text, chunk_size=500, overlap=50):
    if pd.isna(text) or text == '':
        return []

    text = str(text)
    chunks = []

    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]
        if chunk.strip():
            chunks.append(chunk.strip())

    return chunks

In [4]:
def clear_database(tx):
    """데이터베이스의 모든 노드와 관계를 삭제합니다"""
    tx.run("MATCH (n) DETACH DELETE n")

def create_constraints(tx):
    """유니크 제약조건을 생성합니다"""
    constraints = [
        "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Article) REQUIRE a.article_id IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Content) REQUIRE c.content_id IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (m:Media) REQUIRE m.name IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (cat:Category) REQUIRE cat.name IS UNIQUE"
    ]

    for constraint in constraints:
        try:
            tx.run(constraint)
        except Exception as e:
            print(f"제약조건 생성 중 오류: {e}")

with driver.session() as session:
    session.execute_write(clear_database)
    session.execute_write(create_constraints)

In [5]:
def create_article_node(tx, article_data):
    query = """
    MERGE (a:Article {article_id: $article_id})
    SET a.title = $title,
        a.url = $url,
        a.published_date = $published_date
    RETURN a
    """
    tx.run(query,
           article_id=article_data['article_id'],
           title=article_data['title'],
           url=article_data['url'],
           published_date=article_data['published_date'])

def create_content_nodes(tx, article_id, content_chunks, article_data):
    for i, chunk in enumerate(content_chunks):
        content_id = f"{article_id}_chunk_{i}"

        query = """
        MERGE (c:Content {content_id: $content_id})
        SET c.chunk = $chunk,
            c.article_id = $article_id,
            c.title = $title,
            c.url = $url,
            c.published_date = $published_date,
            c.chunk_index = $chunk_index
        """
        tx.run(query,
               content_id=content_id,
               chunk=chunk,
               article_id=article_id,
               title=article_data['title'],
               url=article_data['url'],
               published_date=article_data['published_date'],
               chunk_index=i)

        relationship_query = """
        MATCH (a:Article {article_id: $article_id})
        MATCH (c:Content {content_id: $content_id})
        MERGE (a)-[:HAS_CHUNK]->(c)
        """
        tx.run(relationship_query,
               article_id=article_id,
               content_id=content_id)

def create_media_node_and_relationship(tx, article_id, source):
    if pd.isna(source) or source == '':
        return

    media_query = """
    MERGE (m:Media {name: $source})
    RETURN m
    """
    tx.run(media_query, source=source)

    relationship_query = """
    MATCH (a:Article {article_id: $article_id})
    MATCH (m:Media {name: $source})
    MERGE (m)-[:PUBLISHED]->(a)
    """
    tx.run(relationship_query,
           article_id=article_id,
           source=source)

def create_category_node_and_relationship(tx, article_id, category):
    if pd.isna(category) or category == '':
        return

    category_query = """
    MERGE (cat:Category {name: $category})
    RETURN cat
    """
    tx.run(category_query, category=category)

    relationship_query = """
    MATCH (a:Article {article_id: $article_id})
    MATCH (cat:Category {name: $category})
    MERGE (a)-[:BELONGS_TO]->(cat)
    """
    tx.run(relationship_query,
           article_id=article_id,
           category=category)

In [6]:
def build_graph_from_dataframe(df, chunk_size=500, overlap=50):
    with driver.session() as session:
        for idx, row in df.iterrows():
            try:
                article_id = row.get('article_id', '')

                article_data = {
                    'article_id': article_id,
                    'title': row.get('title', ''),
                    'url': row.get('url', ''),
                    'published_date': str(row.get('published_date', ''))
                }

                # 1. Article 노드 생성
                session.execute_write(create_article_node, article_data)

                # 2. Content 노드들 생성 (content 컬럼이 있는 경우)
                if 'content' in row and pd.notna(row['content']) and row['content'] != '':
                    content_chunks = chunk_text(row['content'], chunk_size, overlap)
                    if content_chunks:  # 청크가 있는 경우에만 생성
                        session.execute_write(create_content_nodes, article_id, content_chunks, article_data)

                # 3. Media 노드와 관계 생성
                if 'source' in row:
                    session.execute_write(create_media_node_and_relationship, article_id, row['source'])
                # 4. Category 노드와 관계 생성
                if 'category' in row:
                    session.execute_write(create_category_node_and_relationship, article_id, row['category'])

                # 진행상황 출력
                if (idx + 1) % 10 == 0:
                    print(f"진행률: {idx + 1}/{len(df)} ({((idx + 1)/len(df)*100):.1f}%)")

            except Exception as e:
                print(f"기사 {idx} 처리 중 오류 발생: {e}")
                continue

In [7]:
with driver.session() as session:
    print("데이터베이스 초기화 중...")
    session.execute_write(clear_database)
    session.execute_write(create_constraints)

build_graph_from_dataframe(df, chunk_size=500, overlap=50)

데이터베이스 초기화 중...
진행률: 10/60 (16.7%)
진행률: 20/60 (33.3%)
진행률: 30/60 (50.0%)
진행률: 40/60 (66.7%)
진행률: 50/60 (83.3%)
진행률: 60/60 (100.0%)
